In [ ]:
import torch
from PIL import Image

IS_GPU = torch.cuda.is_available()

In [ ]:
# Show sample images

In [ ]:
class DDSM(torch.utils.data.Dataset):
    def __init__(self, root, split, transform):
        self.root = root
        self.transform = transform
        self.image_list = []
        
        def process_line(line):
            img_name, label = line.strip().split(' ')
            label = int(label)
            return img_name, label
        
        with open(os.path.join(root, '{}.txt'.format(split)), 'r') as f:
            for line in f.readlines():
                self.image_list.append(process_line(line))
            
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):
        img_name, label = self.image_list[idx]
        img = Image.open(os.path.join(self.root, img_name))
        img = self.transform(img)
        return img, label

In [ ]:
# Dataloader

import os
import torchvision.transforms as transforms

root = '/Users/baohezhang/codes/cs598jp-prj/'

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_dataset = DDSM(root, 'images/train', transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
print(len(train_loader))

In [ ]:
# AlexNet

import torchvision.models as models
import torch.nn as nn

model = models.__dict__['alexnet'](pretrained=True)

model.classifier._modules['6'] = nn.Linear(4096, 2)

if IS_GPU:
    model.features = torch.nn.DataParallel(model.features)
    model.cuda()

model.eval()

In [ ]:
# Train AlexNet
import time

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

epoch_num = 1

start_ts = time.time()

for epoch in range(epoch_num):
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    
    sample_count = 0
    correct_count = 0
    
    for i, samples in enumerate(train_loader):
        images, labels = samples
        if IS_GPU:
            images = images.cuda()
            labels = labels.cuda()
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs.data, 1)
        
        epoch_loss += loss.item()
        
        sample_count += len(labels)
        correct_count += torch.sum(preds == labels).item()
        
        if i % 10 == 0:
            print(i)
    
    print('time: {}'.format(int(time.time() - start_ts)))
    print('epoch {} '.format(epoch) + 'accuracy: ' + str(correct_count / sample_count))

In [ ]:
# Evaluation

with torch.no_grad():
    sample_count = 0
    correct_count = 0
    
    for i, samples in enumerate(train_loader):
        images, labels = samples
        
        if IS_GPU:
            images = images.cuda()
            labels = labels.cuda()
            
        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)
        
        sample_count += len(labels)
        correct_count += torch.sum(preds == labels).item()
    
    print('accuracy: ' + str(correct_count / sample_count))